In [1]:
import pandas as pd
import numpy as np
from imblearn.over_sampling import RandomOverSampler
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_curve, roc_auc_score, classification_report, log_loss
import matplotlib.pyplot as plt

data = pd.read_csv('D:\Liberty 2024 Base\Actual_Data_West.csv')

<>:9: SyntaxWarning: invalid escape sequence '\L'
<>:9: SyntaxWarning: invalid escape sequence '\L'
C:\Users\Admin\AppData\Local\Temp\ipykernel_16408\3079114174.py:9: SyntaxWarning: invalid escape sequence '\L'
  data = pd.read_csv('D:\Liberty 2024 Base\Actual_Data_West.csv')


In [2]:
selected_columns = ['Policy No',  'Renewal Type', 'Product name ', 'Product name  2', 'biztype', 'Policy End Date', 'Policy Start Date', 
                    'Reg no ', 'age', 'MANUFACTURER/Make', 'model', 'variant', 'Fuel Type', 'RTO Location ', 'Before GST Add-on GWP',  
                    'Total OD Premium', 'Total TP Premium', 'gst', 'Total Premium Payable ', 'NCB % Previous Year', 'Vehicle Segment', 
                    'Applicable Discount with NCB', 'New Branch Name  2', 'decline', 'Tie Up', 'Zone 2', 'state2', 'Cleaned_Insured name', 
                    'CustomerID', 'Policy Status', 'Policy Tenure', 'Customer Tenure', 'New Customers', 
                    'Renewal Rate Status', 'Claim in last year', 'Number of Claims']

data = data[selected_columns]

# Convert Policy End Date to datetime
data['Policy End Date'] = pd.to_datetime(data['Policy End Date'], errors='coerce')

# Separate Open Customers (August to October 2024)
open_customers = data[
    (data['Policy Status'].isin(['Renewed', 'Not Renewed'])) &
    (data['Policy End Date'].dt.year == 2024) &
    (data['Policy End Date'].dt.month.isin([7]))
].copy()

# Filter the main dataset for customers whose Policy End Date is <= July 2024
data = data[
    (data['Policy Status'].isin(['Renewed', 'Not Renewed'])) &
    (data['Policy End Date'] <= pd.to_datetime('2024-06-30'))
]

# Map Policy Status to binary for the filtered dataset
data['Policy Status'] = data['Policy Status'].apply(lambda x: 1 if x == 'Not Renewed' else 0)

# Handle missing values
for column in data.columns:
    if data[column].dtype == 'object':
        data[column] = data[column].fillna('missing')
    else:
        data[column] = data[column].fillna(0)

# Extract year, month, and day from date columns
date_columns = ['Policy Start Date', 'Policy End Date']
for col in date_columns:
    data[col] = pd.to_datetime(data[col], errors='coerce')

new_date_cols = {}
for col in date_columns:
    new_date_cols[f'{col}_YEAR'] = data[col].dt.year
    new_date_cols[f'{col}_MONTH'] = data[col].dt.month
    new_date_cols[f'{col}_DAY'] = data[col].dt.day

data = pd.concat([data, pd.DataFrame(new_date_cols)], axis=1)

# Drop original date columns
data = data.drop(columns=date_columns)

# Separate features and target variable for training
features = [col for col in data.columns if col != 'Policy Status']
X = data[features]
y = data['Policy Status']

In [3]:
#label encoding for the actual data
label_encoders = {}
for column in X.columns:
    if X[column].dtype == 'object':
        label_encoder = LabelEncoder()
        X[column] = label_encoder.fit_transform(X[column].astype(str))  
        label_encoders[column] = label_encoder  

C:\Users\Admin\AppData\Local\Temp\ipykernel_16408\2088719358.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[column] = label_encoder.fit_transform(X[column].astype(str))
C:\Users\Admin\AppData\Local\Temp\ipykernel_16408\2088719358.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[column] = label_encoder.fit_transform(X[column].astype(str))
C:\Users\Admin\AppData\Local\Temp\ipykernel_16408\2088719358.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
T

In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, log_loss, roc_auc_score, classification_report, confusion_matrix, roc_curve
import matplotlib.pyplot as plt
import xgboost as xgb

model = xgb.XGBClassifier(
    max_depth=6,
    learning_rate=0.1,
    n_estimators=100,
    random_state=42
)

#Fit the model 
model.fit(X, y)

for col in date_columns:
    if col in open_customers.columns:
        open_customers[col] = pd.to_datetime(open_customers[col], errors='coerce')
    else:
        print(f"Warning: Column {col} not found in open_customers data")

open_customers_new_date_cols = {}
for col in date_columns:
    if col in open_customers.columns:
        open_customers_new_date_cols[f'{col}_YEAR'] = open_customers[col].dt.year
        open_customers_new_date_cols[f'{col}_MONTH'] = open_customers[col].dt.month
        open_customers_new_date_cols[f'{col}_DAY'] = open_customers[col].dt.day

if open_customers_new_date_cols:
    open_customers = pd.concat([open_customers, pd.DataFrame(open_customers_new_date_cols)], axis=1)

open_customers = open_customers.drop(columns=[col for col in date_columns if col in open_customers.columns])

for column in open_customers.columns:
    if open_customers[column].dtype == 'object':
        open_customers[column] = open_customers[column].fillna('missing')
    else:
        open_customers[column] = open_customers[column].fillna(0)

open_customers_encoded = open_customers.copy()

#Label Encoding for open customers
for column in open_customers_encoded.columns:
    if column in label_encoders:  
        encoder = label_encoders[column]

        mapping_dict = {label: i for i, label in enumerate(encoder.classes_)}
        next_unique_value = [max(mapping_dict.values()) + 1]  

        def encode_test_value(value):
            if value in mapping_dict:
                return mapping_dict[value]
            else:
                mapping_dict[value] = next_unique_value[0]
                next_unique_value[0] += 1
                return mapping_dict[value]
        
        open_customers_encoded[column] = open_customers_encoded[column].apply(encode_test_value)

#Predict the open customers
X_open_customers = open_customers_encoded[features]
y_open_pred = model.predict(X_open_customers)
y_open_pred_proba = model.predict_proba(X_open_customers)[:, 1]  

open_customers['Predicted Status'] = ['Not Renewed' if pred == 1 else 'Renewed' for pred in y_open_pred]

print(f"Predicted Renewed: {(y_open_pred == 0).sum()}")
print(f"Predicted Not Renewed: {(y_open_pred == 1).sum()}")

y_pred = model.predict(X)
y_pred_proba = model.predict_proba(X)[:, 1]

#Evaluate the model on training data
train_accuracy = accuracy_score(y, y_pred)
train_log_loss = log_loss(y, y_pred_proba)
train_roc_auc = roc_auc_score(y, y_pred_proba)
train_report = classification_report(y, y_pred)

conf_matrix_train = confusion_matrix(y, y_pred)
class_0_accuracy_train = conf_matrix_train[0, 0] / conf_matrix_train[0].sum()
class_1_accuracy_train = conf_matrix_train[1, 1] / conf_matrix_train[1].sum()

#Print the metrics
print(f"Train Accuracy: {train_accuracy}")
print(f"Train Log Loss: {train_log_loss}")
print(f"Train ROC AUC: {train_roc_auc}")
print(f"Train Classification Report:\n{train_report}")
print(f"Class 0 Train Accuracy: {class_0_accuracy_train}")
print(f"Class 1 Train Accuracy: {class_1_accuracy_train}")

open_customers.to_csv('July_predictions (West).csv', index=False)

Predicted Renewed: 6577
Predicted Not Renewed: 12001
Train Accuracy: 0.7689773839967298
Train Log Loss: 0.45297454792074715
Train ROC AUC: 0.8335164683287002
Train Classification Report:
              precision    recall  f1-score   support

           0       0.67      0.53      0.59     66262
           1       0.80      0.88      0.84    144120

    accuracy                           0.77    210382
   macro avg       0.74      0.70      0.71    210382
weighted avg       0.76      0.77      0.76    210382

Class 0 Train Accuracy: 0.5264404937973499
Class 1 Train Accuracy: 0.8804884818207049
